import sys
!{sys.executable} -m pip install pandas==1.5.2

In [1]:
#import sys
#!{sys.executable} -m pip install html5lib

In [2]:
# import modules/libraries
import warnings 
warnings.simplefilter(action='ignore')
from entsoe import EntsoePandasClient
import pandas as pd
import json
import requests
from pandas import json_normalize
import math
import osmnx as ox
import pandas as pd
import numpy as np
import geopandas as gpd
import time
from datetime import datetime
from scipy import stats
import itertools
import os
import pickle
import geojson
from sqlalchemy import create_engine
import re
import sqlite3
from pathlib import Path
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon
import chardet
from scipy import spatial
from scipy.spatial import KDTree
from shapely import wkt

cwd = Path().resolve()

# visualisation
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
import seaborn as sns
import matplotlib as mpl 
%matplotlib inline 
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

In [3]:
def get_lat_long(name, land_raw):
    """ get latitude and longitude coordinate from POINT geometry """
    if land_raw == 'Stmk':
        bundesland = "Steiermark"
    elif land_raw == 'Sbg':
        bundesland = "Salzburg"
    elif land_raw == 'OÖ':
        bundesland = "Oberösterreich"
    elif land_raw == 'NÖ':
        bundesland = "Niederösterreich"
    else:
        bundesland = land_raw
    text = f"{name}, {bundesland}"
    try:
        #return pd.Series([point.x, point.y])
        data_json = requests.get(url=f'https://nominatim.openstreetmap.org/search?q={text}&format=json&polygon=1&addressdetails=1').json()
        lat = data_json[0]['lat']
        lon = data_json[0]['lon']
        print(text, lat, lon)
        return pd.Series([lat, lon])
            
    except:
        print(text)
        return pd.Series([np.nan, np.nan])
        

def get_verbund_data():
    """ run only once """
    # attrs = {'class': 'wikitable sortable toptextcells zebra jquery-tablesorter'}
    verb = pd.read_html('https://de.wikipedia.org/wiki/Liste_von_Kraftwerken_der_Verbund_AG', skiprows = 0, header=0)[0]
    verb[['lat', 'lon']] = verb.apply(lambda x: get_lat_long(x['Name'], x['Bundesland']), axis=1)
    verb.to_csv('verbund_plants.csv')

    
def get_sag_data():
    """ run only once """
    sag = pd.read_html('http://www.salzburg-ag.at/ueber-die-salzburg-ag/unternehmen/erzeugung/erzeugungsanlagen.html',
                displayed_only=True)
    sag.to_csv('sag_plants.csv')
    

In [4]:
plants = pd.read_csv('verbund_plants.csv')
plants['Betreiber'] = "Verbund"
plants.head()

,Unnamed: 0,Name,Typ,Gewässer,Land,Bundesland,Leistung in MW,lat,lon,Betreiber
0,0,Abwinden-Asten,Laufwasserkraftwerk,Donau,AT,OÖ,1680,48.247645,14.430105,Verbund
1,1,Altenmarkt,Laufwasserkraftwerk,Enns,AT,Stmk,210,46.772845,15.540325,Verbund
2,2,Altenwörth,Laufwasserkraftwerk,Donau,AT,NÖ,3280,48.384500,15.859538,Verbund
3,3,Annabrücke,Laufwasserkraftwerk,Drau,AT,Kärnten,900,46.572964,14.491360,Verbund
4,4,Arnstein,Speicherkraftwerk,Teigitsch,AT,Stmk,300,47.026680,15.170987,Verbund


In [5]:
def load(filename='sag_plants.json'):
    """ run only once -> it is saved to csv for further processing, in case website is offline"""
    f = open(f'{filename}')
    data = json.load(f)
    f.close()
    df = json_normalize(data)
    df.rename(columns={'title' : 'Name', 'latitude':'lat', 'longitude':'lon'}, inplace=True)
    return df


def get_power_data(path):
    # print(path)
    table = pd.read_html(f'http://www.salzburg-ag.at{path}', skiprows=1, displayed_only=True)[0]
    print(table.head())
    return pd.Series([table.Inbetriebnahme])

df = load()
#df[['Inbetriebnahme']] = df.apply(lambda x: get_power_data(x['path']), axis=1)
df.head()

,Name,lon,lat,image,mapType,tableItems,path,mapTypeLabel,region
0,HKW Nord Power to heat,13.033926,47.827775,/content/dam/web18/bilder/gebaeude/kraftwerke/...,heizkraftwerk,[],/ueber-die-salzburg-ag/unternehmen/erzeugung/e...,Heiz(kraft)werk,Stadt Salzburg
1,PV-Anlage GeschÃ¤ftsstelle Golling,13.180723,47.591751,NaN,photovoltaikanlage,[],/ueber-die-salzburg-ag/unternehmen/erzeugung/e...,Photovoltaikanlage,Tennengau
2,HKW Mitte Power to heat,13.038196,47.809199,NaN,heizkraftwerk,[],/ueber-die-salzburg-ag/unternehmen/erzeugung/e...,Heiz(kraft)werk,Stadt Salzburg
3,PV-Anlage Zentrallager Salzburg AG,13.051664,47.813101,NaN,photovoltaikanlage,[],/ueber-die-salzburg-ag/unternehmen/erzeugung/e...,Photovoltaikanlage,Stadt Salzburg
4,PV-Anlage Bahnhof Lamprechtshausen,12.951766,47.990422,NaN,photovoltaikanlage,[],/ueber-die-salzburg-ag/unternehmen/erzeugung/e...,Photovoltaikanlage,Flachgau


In [6]:
boundary_geojson = gpd.read_file(os.path.join(Path(cwd), 'laender_95_topo.json'))
#boundary_geojson.drop(columns=['cartodb_id', 'created_at', 'updated_at'], inplace=True)
#region = boundary_geojson.geometry.unary_union

In [7]:
boundary_geojson.head()

,iso,name,geometry
0,4,Oberösterreich,"POLYGON ((13.76147 47.52081, 13.74994 47.50213..."
1,7,Tirol,"MULTIPOLYGON (((12.71233 46.73610, 12.71151 46..."
2,6,Steiermark,"POLYGON ((14.07995 47.02247, 14.07312 47.01310..."
3,1,Burgenland,"POLYGON ((16.06678 46.85025, 16.06004 46.83935..."
4,3,Niederösterreich,"POLYGON ((14.85902 47.74099, 14.84094 47.73446..."


In [15]:
def get_geo_data():
    """ Load Austrian geojson"""
    with open(os.path.join(Path(cwd), 'laender_95_topo.json'), encoding='utf8') as a:
        counties = json.load(a)
    return counties

geojson=get_geo_data()
inverse_geojson = geojson
#j= inverse_geojson['features'][1]['geometry']['coordinates']
insertt = [[-180, -90], [-180, 90], [180, 90], [180, -90], [-180, -90]] #[[0,90],[180,90],[180,-90],[0,-90],[-180,-90],[-180,0],[-180,90],[0,90]]
#j[0].insert(0, insertt)
#j[0]
for idx, i in enumerate(geojson['features']):
    l= [i['geometry']['coordinates']]
    #print(l[0])
    l[0].insert(0, insertt)
    #print(l[0])
    inverse_geojson['features'][idx]['geometry']['coordinates'] = l[0]
#print(inverse_geojson)

In [16]:


#geojson=get_geo_data()
#inverse_geojson = geojson
#j= geojson['features'][1]['geometry']['coordinates']
#insertt = [[-180, -90], [-180, 90], [180, 90], [180, -90], [-180, -90]] #[[0,90],[180,90],[180,-90],[0,-90],[-180,-90],[-180,0],[-180,90],[0,90]]
#j[0].insert(0, insertt)
#j[0]
#inverse_geojson['features'][0]['geometry']['coordinates'] = j[0]


In [24]:
d = {'val': [1,2,3], 'namee': ['Salzburg', 'Oberösterreich', 'Kärnten'], 'iso': [5, 4,3]}
dfff = pd.DataFrame(data=d, index=[0,1,2])
dfff.val = dfff.val.astype('category')
df3=dfff[dfff['namee']=='Salzburg']
df3.head()

,val,namee,iso
0,1,Salzburg,5


In [25]:
fig = px.choropleth_mapbox(df3, geojson=inverse_geojson, locations='namee', featureidkey="properties.name", color='val',
                          mapbox_style="carto-positron",
                               zoom=6, center={"lat": 47.809490, "lon": 13.055010},)
fig.show()

In [26]:
import plotly.express as px
import plotly.graph_objects as go

#data = pd.read_excel('tmp.xlsx', index_col=0,
#              na_values=['string1', 'string2'])  
#data = pd.read_csv('kraftwerke_salzburg.csv', sep=",", index_col=False)  
# plants = t[t['Bundesland'] == "Sbg"]
def add_trace(fig, df, remark, col, size):
   # if remark == "verbund":
   #     template=f"<br>Name: {df['Name']}<br>Typ: {df['Typ']}<br>Leistung: {df['Leistung in MW']} MW<br>Betreiber: {df['Betreiber']}"
   # elif remark == "sag":
    #    template=f"<br>Name: {df['Name']}"
        
    fig.add_scattermapbox(
        lat=[df['lat']],
        lon=[df['lon']],
        #marker_symbol=marker,
        mode='markers', #'markers+text'
        #text=texts,
        #marker = marker,
        marker_size=size,
        marker_color=col,
        line=dict(width=12),
        
        opacity= 0.5,
        #hovertemplate= template,
        showlegend=False,
       # hover_data=['amenity'],
       # hoverinfo=all
      #  label={'trace 1':'test'},
       # name=name
    )



def translate(value, leftMin, leftMax, rightMin, rightMax):
    # Figure out how 'wide' each range is
    leftSpan = leftMax - leftMin
    rightSpan = rightMax - rightMin
    # Convert the left range into a 0-1 range (float)
    valueScaled = float(value - leftMin) / float(leftSpan)
    # Convert the 0-1 range into a value in the right range.
    return rightMin + (valueScaled * rightSpan)




In [28]:
def kraftwerke_salzburg():
    #fig = go.Figure()
    fig = px.choropleth_mapbox(df3, geojson=inverse_geojson, locations='namee', featureidkey="properties.name", color='val',
                          mapbox_style="carto-positron", color_discrete_sequence=['#FECB52'],
                               zoom=6, center={"lat": 47.809490, "lon": 13.055010},)
    
    """
    go.Scattermapbox(mode='markers',lat=plants.lat.tolist(), lon=plants.lon.tolist(), marker={'size': 10}, 
                                    # hoverinfo='skip',
                                     text = plants.Name.tolist(),
                                  # hover_name="ort",
                                     showlegend=False)
                                     """
    for index, row in plants.iterrows():
        add_trace(fig, row, 'verbund', "#76b5c5", translate(row['Leistung in MW'], plants['Leistung in MW'].min(), plants['Leistung in MW'].max(),
                                                     10, 30))
    
    for index, row in df.iterrows():
        add_trace(fig, row, 'sag', "#e28743", 9)
    
    
    #fig.update_layout(mapbox_style="light", mapbox_accesstoken="pk.eyJ1IjoiYXJlYnVyZyIsImEiOiJjbDZkZzdhOXAwNTk5M2ZteDdkY3lvd2FsIn0.c4hsFv69HozIVHq7ItrTzg", mapbox_zoom=6,
    #                  mapbox_center={"lat": 48.210033, "lon": 16.363449})
    #fig.update_layout(font=dict(family="Helvetica"), legend={"title": "Select category"})
    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0}, paper_bgcolor='rgba(0,0,0,0)',
                  plot_bgcolor='rgba(0,0,0,0)')

    fig.show()

kraftwerke_salzburg()